In [228]:
import pandas as pd
import numpy as np
import os

In [229]:
# Apenas rodar esse notebook após capturar resultados dos modelo de AI Gen que estará em "./results"

In [230]:
results = sorted(os.listdir("./result"), key=lambda x: int(x.split("_")[1]))

In [231]:
df = pd.DataFrame()

In [232]:
df = pd.DataFrame()

for i, path in enumerate(results):

    df_result_range = pd.read_csv(f"result/{path}" ,sep =";")

    if (i+1) % 14 == 0:
        df_result_range = df_result_range.iloc[:5,:]

    df = pd.concat([df, df_result_range])

In [233]:
df

,number,classification
0,1,1
1,2,1
2,3,1
3,4,2
4,5,2
...,...,...
0,196,1
1,197,1
2,198,1
3,199,1


In [234]:
real_label_eggs = pd.read_excel("./IAGenOvoscopia/OVOSCOPIA-1RODADA - Sem 28dias.xlsx")
real_label_eggs.columns = ["N_OVO", "OVOSCOPIA", 'TEMPO', 'observações']

In [235]:
df = df.rename(columns={'number': 'N_OVO', 'classification': 'predict'})
#df['N_OVO'] = range(1,801)
df['N_OVO'] = range(1, len(df) + 1)


In [236]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [237]:
final_df = real_label_eggs.merge(df)

In [238]:
print(final_df.columns)


Index(['N_OVO', 'OVOSCOPIA', 'TEMPO', 'observações', 'predict'], dtype='object')


In [239]:
cf_matrix = confusion_matrix(final_df['OVOSCOPIA'], final_df['predict'])

In [240]:
report = classification_report(final_df['OVOSCOPIA'], final_df['predict'])
print(report)

              precision    recall  f1-score   support

           1       0.74      0.91      0.82       108
           2       0.83      0.62      0.71        81
           3       0.83      0.56      0.67         9
           4       1.00      1.00      1.00         2

    accuracy                           0.78       200
   macro avg       0.85      0.77      0.80       200
weighted avg       0.79      0.78      0.77       200



In [257]:
analise_df = pd.read_excel("./analise.xlsx")

In [260]:
report = classification_report(final_df['OVOSCOPIA'], final_df['predict'], output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.reset_index(inplace=True)
report_df.rename(columns={'index': 'classification'}, inplace=True)


last_index = analise_df['prompt'].last_valid_index()
start_row = last_index + 2 if last_index is not None else 0

while start_row + len(report_df) > len(analise_df):
    new_row = pd.Series([None] * len(analise_df.columns), index=analise_df.columns)
    analise_df = pd.concat([analise_df, new_row.to_frame().T], ignore_index=True)

for col in ['classification', 'precision', 'recall', 'f1-score', 'support']:
    if col in analise_df.columns:
        analise_df.loc[start_row:start_row + len(report_df) - 1, col] = report_df[col].values

analise_df.to_excel("./analise.xlsx", index=False)

In [261]:
print(analise_df)


                                               prompt classification  \
0   \n    Analise as imagens, e classifique usando...            NaN   
1   \n    Analise as imagens, e classifique usando...              1   
2                                                 NaN              2   
3                                                 NaN              3   
4                                                 NaN              4   
5                                                 NaN       accuracy   
6                                                 NaN      macro avg   
7                                                 NaN   weighted avg   
8                                                 NaN            NaN   
9                                                 NaN            NaN   
10  Analise as imagens, e classifique usando a esc...              1   
11                                                NaN              2   
12                                                NaN           